### Breaking out of Nested Loops

Sometimes, when we have nested loops, we need to break out of the outermost loop while the code is running in the innermost loop.

Unfortunately Python does not provide a language mechanism for doing this, unlike some other languages such as Rust.

Let's look at an example.

Let's say we have a matrix such as this:

$$
\begin{bmatrix}
0 & 0 & 1 & 2 \\
0 & 1 & 3 & 4 \\
0 & 5 & 10 & -2 \\
-1 & 2 & 12 & -4
\end{bmatrix}
$$

Our goal is to find the value and row and column index of the first element (moving from left to right and top to bottom), that is negative.

In Python, we might represent such a matrix as list of lists as follows:

In [1]:
m = [
    [0, 0, 1, 2],
    [0, 1, 3, 4],
    [0, 5, 10, -2],
    [-1, 2, 12, -4]
]

In this case, the answer we are looking for would the value -2, located at row index 2, column index 3.

To find this element we could use a nested loop, but when we find the element in the inner loop, we need to break out of **both** loops - unfortunately, `break` only breaks out of the current loop the code block is running in.

In [2]:
for row_index, row in enumerate(m):
    for column_index, value in enumerate(row):
        if value < 0:
            print(f"Found negative element: {value}, at position ({row_index},{column_index})")
            break

Found negative element: -2, at position (2,3)
Found negative element: -1, at position (3,0)


As you can see, we need to break out of both loops.

#### Approach 1

We could use a flag to track if the outer loop needs to break.

In [3]:
break_out = False
for row_index, row in enumerate(m):
    if break_out:
        break
    for column_index, value in enumerate(row):
        if value < 0:
            print(f"Found negative element: {value}, at position ({row_index},{column_index})")
            break_out = True
            break

Found negative element: -2, at position (2,3)


While this certainly works, it is not very elegant code - we have that flag we have to keep track of, and someone reading this code is going to have to think a bit to understand what we are doing - that's what I call inelegant.

#### Approach 2

Is there a better approach?

Maybe.

Remember the `else` clause that `for` loops have?

Yeah, if you're like most people, you probably don't use it, and either never learned it, or forgot it!

Let's go over how that clause works:

In [4]:
for i in range(6):
    print(i)
    if i > 3:
        break
else:
    print("Else clause executed")


0
1
2
3
4


As you can see that `else` block attached to the `for` loop never executed.

Now let's look at this:

In [5]:
for i in range(6):
    print(i)
else:
    print("Else clause executed")

0
1
2
3
4
5
Else clause executed


Aha - so the `else` clause executes if the `for` loop **completes without a** `break`.

Since that is a bit confusing, I always add a comment this way, which helps me remember how the `else` works:|

In [6]:
for i in range(6):
    print(i)
    if i > 3:
        break
else:  # nobreak
    print("Else clause executed")

0
1
2
3
4


So now, we could use this to our advantage in the previous example:

In [7]:
for row_index, row in enumerate(m):
    for column_index, value in enumerate(row):
        if value < 0:
            print(f"Found negative element: {value}, at position ({row_index},{column_index})")
            break
    else:  # nobreak
        # inner loop completed without breaking - so we did not find our element
        continue
    
    # if we reach this line, means that the inner loop **did** break
    # so we need to break out of the current loop (the outer loop)
    break

Found negative element: -2, at position (2,3)


Removing all the comments we have:

In [8]:
for row_index, row in enumerate(m):
    for column_index, value in enumerate(row):
        if value < 0:
            print(f"Found negative element: {value}, at position ({row_index},{column_index})")
            break
    else:  # nobreak 
        continue
    break

Found negative element: -2, at position (2,3)


Ok, so that works too - it is certainly more Pythonic than our first approach, but I still find it somewhat confusing code - I don't particularly like it much either, but I still prefer it to approach #1.

#### Approach 3

Remember these things called **exceptions**?

Although you may only think of them as used in case of "errors", they are indeed for more useful than that - exceptions are actually a form of **flow control**.

For example, that's how `for` loops know when to stop iterating over an iterator - the iterator raises a `StopIteration` exception when the iterator is done producing results. This is not an error state, simply a statement of the fact that the iterator has been exhausted, and the `for` loop is then informed that iteration is complete - that's flow control!

So, we could use the same mechanism to indicate to the outer loop that the inner loop found what it needs, and that the outer loop can stop iterating.

We could use a custom exception here:

In [9]:
class FoundElement(Exception):
    pass

And code it this way:

In [10]:
for row_index, row in enumerate(m):
    try:
        for column_index, value in enumerate(row):
            if value < 0:
                print(f"Found negative element: {value}, at position ({row_index},{column_index})")
                raise FoundElement
    except FoundElement:
        break

Found negative element: -2, at position (2,3)


But why create a needless custom exception for this - we can just use one of the built-in exceptions, and `StopIteration` seems like a good choice to me here:

In [11]:
for row_index, row in enumerate(m):
    try:
        for column_index, value in enumerate(row):
            if value < 0:
                print(f"Found negative element: {value}, at position ({row_index},{column_index})")
                raise StopIteration
    except StopIteration:
        break

Found negative element: -2, at position (2,3)


So far, this is the solution I like the best. But...

#### Approach 4

I still don't particularly love any of the above solutions.

When I see coding patterns like the one above, I'm thinking that basically we want to **return** the first element we find that matches the criteria.

To me that sounds like exactly what a **function** would do.

And, in fact, if the code above were included in some larger chunk of code, we would also be simplifying the larger chunk of code by decomposing our problem further.

So now, we're going to write a function to simply return what we need:

In [12]:
def find_first_negative_element(matrix):
    for row_index, row in enumerate(m):
        for column_index, value in enumerate(row):
            if value < 0:
                return row_index, column_index, value
    return (None,) * 3  # keeps the return consistent if nothing was found

And then in our code we could use this function:

In [13]:
# larger code block
#...
row_index, column_index, value = find_first_negative_element(m)
if row_index is not None:
    print(f"Found negative element: {value}, at position ({row_index},{column_index})")

Found negative element: -2, at position (2,3)


And that, to me, is the **best** solution - not only does it avoid all the complexity of breaking out of multiple levels of nested loops, but it also simplifies our code, makes it far more readable, and makes it easier to unit test too!

#### Other Solutions

I'm sure there are other clever ways of doing this that I haven't though of here, but I doubt their simplicity will be as good as the function approach we saw in #4. Remember too, that you would need to look for a technique that allows you to break out of multiple nested loops in general, not just the two nested loops we used in our example. The function approach will work equally simply for those cases, whereas the other ones just require piling on more and more control code (with the exception of the Exception based approach in #3, where you would just catch the `StopIteration` in the outermost loop, and let the exception propogate from the innermost loop.